# Solving Logistic Problems with ORS and or-tools

The logistics of delivering physical items as efficiently as possible has always been of huge interest for businesses. With the rise of online shopping more and more packages are delivered every day to even private consumers. Therefore, making delivery services more effective is a lucrative field for companies all over the world.

Here, we want to showcase an easy solution to generic logistic problems using [**or-tools**](https://github.com/google/or-tools) and the [**openrouteservice API**](https://openrouteservice.org/). The first step is to import the required packages. 

In [1]:
from IPython.core.display import display, HTML
import openrouteservice
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import folium
from shapely import wkt, geometry
import numpy as np 
import pandas as pd

# Setting up the problem

Let say we a are small delivery company in Heidelberg and today we have 21 customers asking us to deliver their package orders.  Of course every client is ordering something different and hence the package sizes vary. We quantify the package size by a demand number. A higher demand number means a bigger package needs to be delivered. As a small company, we only have 4 delivery trucks and each of them can only transport a maximum demand of 100. Our goal is to deliver the packages as fast as possible.  

In [2]:
# position, demand and address names of the 21 customers + our company (with demand 0)
CVRP = np.array([[  8.683448,  49.416961,  0,   'Quinckestraße 49'            ],
                 [  8.681247,  49.412388,  24,  'Bergfriedhof'                ],
                 [  8.662789,  49.37446,   6,   'Gertrude-von-Ubisch-Straße'  ],
                 [  8.663488,  49.390504,  20,  'Speyerer Straße'             ],
                 [  8.650365,  49.424738,  9,   'Klostergasse 6'              ],
                 [  8.684574,  49.393819,  19,  'Feuerbachstraße 26'          ],
                 [  8.636446,  49.430018,  11,  'A5'                          ],
                 [  8.693395,  49.401669,  22,  'Mittlerer Gaisbergweg 11'    ],
                 [  8.685109,  49.387783,  18,  'Eisenhower Street'           ],
                 [  8.670162,  49.406377,  6,   'Czernyring 10'               ],
                 [  8.63245,   49.409997,  23,  'Handelsstraße 1'             ],                
                 [  8.689162,  49.372064,  21,  'Traitteurweg'                ],
                 [  8.670682,  49.396678,  3,   'Baumschulenweg'              ],
                 [  8.690306,  49.372096,  28,  'Schleifweg 44'               ],
                 [  8.69358,   49.378182,  13,  'Oelgasse 1a'                 ],
                 [  8.63606,   49.395647,  4,   'Rudolf-Diesel-Straße 20'     ],
                 [  8.670398,  49.408443,  26,  'Yorckstraße'                 ],
                 [  8.637578,  49.429881,  25,  'Am Taubenfeld 35'            ],
                 [  8.682342,  49.403657,  2,   'Ringstraße 19a'              ],
                 [  8.64455,   49.413006,  4,   'Wieblinger Weg 100a'         ],
                 [  8.685483,  49.406815,  15,  'Alte Glockengießerei'        ],
                 [  8.691885,  49.372719,  11,  'Hermann-Schück-Weg 1'        ]])

df_CVRP = pd.DataFrame(CVRP, columns=['longitude', 'latitude', 'demand', 'address'])
df_CVRP.columns.name = 'node number'
display(df_CVRP)

###########################
# Problem Data Definition #
###########################

locations = CVRP[:, :2].astype(float) 
demands = CVRP[:, 2].astype(int) 
names = CVRP[:, 3]
num_locations = len(locations)
depot = 0    # The depot is the start and end point of each route aka our company domicile
num_vehicles = 4
capacity = 100

node number,longitude,latitude,demand,address
0,8.683448,49.416961,0,Quinckestraße 49
1,8.681247,49.412388,24,Bergfriedhof
2,8.662789,49.37446,6,Gertrude-von-Ubisch-Straße
3,8.663488,49.390504,20,Speyerer Straße
4,8.650365,49.424738,9,Klostergasse 6
5,8.684574,49.393819,19,Feuerbachstraße 26
6,8.636446,49.430018,11,A5
7,8.693395,49.401669,22,Mittlerer Gaisbergweg 11
8,8.685109,49.387783,18,Eisenhower Street
9,8.670162,49.406377,6,Czernyring 10


To get a rough idea where all the packages need to be delivered to, a quick plot will give some insight:

In [3]:
# coordinates of the polygon borders of Heidelberg
poly_coords = [(49.4157627, 8.5734302), (49.416518, 8.573288), (49.4169018, 8.5732227), (49.4172172, 8.5732061), (49.4175624, 8.5732105), (49.4180262, 8.5732329), (49.4182387, 8.5732441), (49.4184603, 8.5732621), (49.418634, 8.5732802), (49.4191025, 8.5732535), (49.4198517, 8.5734867), (49.4202081, 8.5735754), (49.4204778, 8.5736183), (49.4207656, 8.5736296), (49.4210285, 8.5736023), (49.4217462, 8.5735169), (49.4232662, 8.5732443), 
               (49.4235999, 8.5731788), (49.4240214, 8.5742425), (49.4244224, 8.5752902), (49.4243563, 8.5753853), (49.4250846, 8.5772883), (49.4252735, 8.5777912), (49.4261985, 8.5798331), (49.4264503, 8.5813791), (49.4264726, 8.582023), (49.4265061, 8.5829195), (49.4265415, 8.5835079), (49.4265995, 8.5840786), (49.4268456, 8.5862321), (49.4269339, 8.5870795), (49.4269574, 8.5872399), (49.4270379, 8.5877643), (49.427151, 8.5884423), 
               (49.4272957, 8.589435), (49.4273293, 8.5897569), (49.4273469, 8.5900544), (49.4273672, 8.5904498), (49.4277347, 8.5907782), (49.4301327, 8.5929629), (49.4300245, 8.5932586), (49.428143, 8.5982295), (49.4280444, 8.5984896), (49.4285075, 8.5983822), (49.4279717, 8.5998844), (49.4258258, 8.6054724), (49.4242417, 8.6097328), (49.4243821, 8.6099518), (49.4278838, 8.6153096), (49.4283026, 8.6159738), (49.4286207, 8.6162598), 
               (49.4322642, 8.6194931), (49.4338927, 8.6209266), (49.4338425, 8.6210495), (49.4339638, 8.6211644), (49.4342246, 8.6213947), (49.435144, 8.622204), (49.4352789, 8.6223085), (49.4354116, 8.6224062), (49.4355946, 8.6225211), (49.43575, 8.6226186), (49.4359146, 8.6227197), (49.4361021, 8.6228311), (49.4362255, 8.6229251), (49.4363558, 8.6230368), (49.4366312, 8.62331), (49.4393941, 8.6261342), (49.439998, 8.6267425), 
               (49.4400589, 8.6266155), (49.4402646, 8.626819), (49.4408265, 8.6273509), (49.4415698, 8.6280122), (49.4419758, 8.6282108), (49.4431153, 8.6287031), (49.4430182, 8.6293672), (49.4428791, 8.6304526), (49.4427199, 8.6316013), (49.442524, 8.6327017), (49.4423421, 8.6338579), (49.4421257, 8.6349833), (49.4418929, 8.6360774), (49.4416306, 8.6371722), (49.4413357, 8.6382116), (49.4409796, 8.6393045), (49.4406068, 8.6403103), 
               (49.4402041, 8.6412921), (49.4397852, 8.6422462), (49.4396349, 8.6425644), (49.4396096, 8.6426096), (49.4393088, 8.6431488), (49.4389438, 8.643741), (49.4386231, 8.6441677), (49.4381873, 8.6447541), (49.4378514, 8.6451587), (49.4376043, 8.6454877), (49.4373455, 8.6457854), (49.437132, 8.6460161), (49.4369732, 8.6461932), (49.4366256, 8.6465517), (49.4368784, 8.6472182), (49.437252, 8.6482036), (49.4376933, 8.6489683), 
               (49.4377847, 8.6488619), (49.4378655, 8.6490535), (49.4393174, 8.6533943), (49.4399428, 8.6553099), (49.4386056, 8.656308), (49.4387578, 8.6570031), (49.4390386, 8.6582674), (49.4388724, 8.6583852), (49.4392854, 8.6604604), (49.4395384, 8.6617348), (49.4396636, 8.6623738), (49.4396889, 8.6624386), (49.4396856, 8.6626366), (49.4396297, 8.6630463), (49.4398572, 8.6632376), (49.4399474, 8.6632449), (49.4399409, 8.6634935), 
               (49.4402266, 8.6652141), (49.4402665, 8.6653439), (49.4402759, 8.6654033), (49.4402998, 8.6654508), (49.440349, 8.6655156), (49.4405364, 8.6656772), (49.4406398, 8.6658483), (49.4407718, 8.6662637), (49.4408355, 8.6671038), (49.4412616, 8.6689318), (49.4414666, 8.6699148), (49.4414938, 8.67033), (49.441343, 8.6707607), (49.4408339, 8.6715745), (49.4411247, 8.672341), (49.4413263, 8.6727404), (49.4412527, 8.6728605), 
               (49.441504, 8.6732929), (49.4419681, 8.6747767), (49.4420352, 8.6751848), (49.4421986, 8.6760966), (49.4423485, 8.676933), (49.4423913, 8.6770124), (49.4424941, 8.6772067), (49.4426242, 8.6783343), (49.4425922, 8.6783587), (49.4426978, 8.679192), (49.4430196, 8.6791132), (49.4431327, 8.67967), (49.4434827, 8.6803928), (49.4435342, 8.6807956), (49.4436749, 8.6812441), (49.4436923, 8.6816573), (49.4437735, 8.6821161), 
               (49.4438621, 8.6823346), (49.4439709, 8.682677), (49.444, 8.6829642), (49.4438779, 8.6834956), (49.4437677, 8.6838872), (49.4436709, 8.6842997), (49.4435404, 8.6846072), (49.4433548, 8.6849598), (49.4433286, 8.6854463), (49.4433193, 8.6855722), (49.4432134, 8.6860652), (49.4430476, 8.6867993), (49.4430076, 8.6873627), (49.4430345, 8.6875692), (49.4432118, 8.6878572), (49.4432628, 8.6893699), (49.4432361, 8.6904962), 
               (49.4436499, 8.6912439), (49.4438452, 8.6917068), (49.4440175, 8.6921662), (49.4441168, 8.6927337), (49.4442059, 8.6936862), (49.4445855, 8.6955151), (49.4445549, 8.6959665), (49.4446656, 8.6964361), (49.4448655, 8.6968641), (49.4450251, 8.6980094), (49.4454739, 8.7004409), (49.4458191, 8.7022347), (49.4459197, 8.7032258), (49.4460002, 8.7039892), (49.4460219, 8.7044864), (49.4459241, 8.7053785), (49.4458723, 8.7060433), 
               (49.4458571, 8.7067328), (49.4458904, 8.7075268), (49.4459478, 8.7080621), (49.4461562, 8.7089577), (49.4461977, 8.709325), (49.4466472, 8.7094119), (49.4475009, 8.7094671), (49.4477245, 8.7095193), (49.4490125, 8.7106972), (49.4494832, 8.7113827), (49.4498477, 8.7117118), (49.4500623, 8.7119227), (49.4501811, 8.7120631), (49.4502632, 8.7122212), (49.4506354, 8.7126181), (49.4511883, 8.7129072), (49.4513369, 8.7129813), 
               (49.4514144, 8.7130796), (49.4515467, 8.7134478), (49.4517635, 8.7137288), (49.4519643, 8.7140272), (49.4521719, 8.7143957), (49.4523383, 8.7148131), (49.4524202, 8.7151741), (49.452383, 8.7156469), (49.4522906, 8.716368), (49.4521868, 8.7169946), (49.4521176, 8.7174147), (49.4520394, 8.7178277), (49.4518285, 8.7183244), (49.4514823, 8.7189887), (49.4510654, 8.7196141), (49.4507447, 8.7200859), (49.4505452, 8.7204913), 
               (49.4502382, 8.7210227), (49.4498146, 8.7214415), (49.4495034, 8.7216961), (49.4493455, 8.7218531), (49.449185, 8.7220774), (49.4490165, 8.722459), (49.4488205, 8.7228966), (49.4485791, 8.7236352), (49.4484516, 8.7241952), (49.448206, 8.7254378), (49.4482134, 8.7264384), (49.4483732, 8.7300149), (49.4485688, 8.7327478), (49.44883, 8.7346234), (49.4489557, 8.7351517), (49.4491272, 8.7357989), (49.4492417, 8.7365408), 
               (49.4492374, 8.7368523), (49.4491713, 8.7370834), (49.4490596, 8.7373179), (49.4484233, 8.7380076), (49.4480172, 8.7384311), (49.4469404, 8.7392122), (49.4463859, 8.7392055), (49.4468911, 8.7456999), (49.4474185, 8.7457418), (49.4481419, 8.7458535), (49.4483792, 8.7458079), (49.4486737, 8.7457483), (49.4488244, 8.7456811), (49.4490853, 8.745437), (49.449477, 8.7450254), (49.4497608, 8.744778), (49.4500331, 8.7447195), 
               (49.4502776, 8.7447239), (49.4504288, 8.7445564), (49.4506847, 8.7447186), (49.4510228, 8.7448706), (49.4519911, 8.7455857), (49.4526261, 8.7459664), (49.4532339, 8.7462352), (49.4534992, 8.7462012), (49.4537006, 8.746118), (49.4541602, 8.7460253), (49.4545215, 8.7459427), (49.4546909, 8.7459117), (49.4547708, 8.7459753), (49.4548963, 8.7460633), (49.455054, 8.7460778), (49.4553628, 8.7460827), (49.4556418, 8.7460524), 
               (49.455754, 8.745895), (49.4559743, 8.7453881), (49.4560822, 8.7451013), (49.4560892, 8.7449542), (49.456069, 8.7447755), (49.4561012, 8.7446284), (49.4561678, 8.7444362), (49.4562827, 8.7440512), (49.4563521, 8.7435576), (49.4563346, 8.7430496), (49.4563285, 8.7425696), (49.4564001, 8.7420481), (49.4564782, 8.7418417), (49.4565627, 8.741807), (49.4570407, 8.7417459), (49.4575576, 8.7416743), (49.4577697, 8.7419695), 
               (49.4580796, 8.7426433), (49.4583052, 8.7431205), (49.4583784, 8.74324), (49.4584972, 8.7432195), (49.4586479, 8.7433358), (49.4591587, 8.7443151), (49.4594344, 8.7448837), (49.4595711, 8.7452346), (49.459614, 8.7454695), (49.459567, 8.7463521), (49.4594413, 8.7478334), (49.459252, 8.7489921), (49.4590719, 8.7500913), (49.458958, 8.7504381), (49.4592042, 8.75158), (49.4595483, 8.7530987), (49.4596926, 8.7539863), 
               (49.4596418, 8.7547049), (49.4594683, 8.7546861), (49.4588537, 8.7554084), (49.4584054, 8.756062), (49.4579337, 8.756618), (49.4574727, 8.7570405), (49.4567018, 8.7575548), (49.4566621, 8.7586337), (49.4565725, 8.7605502), (49.4564876, 8.7629394), (49.4564667, 8.7632897), (49.4563679, 8.7635746), (49.4555467, 8.7653324), (49.4541924, 8.7677061), (49.4536984, 8.7671066), (49.4531027, 8.7663194), (49.4524693, 8.7652773), 
               (49.4518931, 8.7642763), (49.4514327, 8.7631687), (49.4509471, 8.7620511), (49.4508064, 8.7618183), (49.4506181, 8.7617028), (49.4481354, 8.7617663), (49.4460471, 8.7647278), (49.4435839, 8.7663344), (49.4418633, 8.7663702), (49.4402744, 8.7662227), (49.4402744, 8.7665154), (49.4373654, 8.7663982), (49.4360464, 8.7694818), (49.4357373, 8.7704502), (49.4355359, 8.7721699), (49.4354634, 8.7729228), (49.435419, 8.7732522), 
               (49.4353177, 8.7736413), (49.4349928, 8.7740221), (49.4345399, 8.7749916), (49.434071, 8.7759857), (49.4333573, 8.7767651), (49.433155, 8.7769731), (49.4330122, 8.7772682), (49.4329069, 8.777773), (49.432764, 8.7792962), (49.4325778, 8.7865372), (49.4311463, 8.7883401), (49.4300674, 8.7890697), (49.4270798, 8.7896433), (49.4259445, 8.7899778), (49.4240512, 8.790903), (49.4232584, 8.7912725), (49.4228803, 8.7911708), 
               (49.4212125, 8.7901624), (49.4210411, 8.7899637), (49.4205773, 8.7898191), (49.4198308, 8.789554), (49.4192557, 8.7893938), (49.4188175, 8.7894293), (49.4176078, 8.7894795), (49.416379, 8.7899429), (49.4152862, 8.7901317), (49.4149639, 8.7901989), (49.4147822, 8.7903774), (49.4123489, 8.7923589), (49.4118527, 8.792762), (49.4116468, 8.792821), (49.4110431, 8.7929421), (49.410423, 8.7930724), (49.4099483, 8.7931202), 
               (49.4089092, 8.7934534), (49.4071899, 8.7940495), (49.4069305, 8.7939641), (49.4063504, 8.793701), (49.4060631, 8.7936568), (49.4056335, 8.7937081), (49.4051869, 8.7935909), (49.4050042, 8.7933146), (49.4046752, 8.7927557), (49.4042374, 8.7920458), (49.4037343, 8.7914362), (49.4030682, 8.7906995), (49.4027096, 8.790122), (49.402355, 8.7892438), (49.4017802, 8.7874791), (49.4015321, 8.7864496), (49.4013797, 8.7854261), 
               (49.4013288, 8.7846816), (49.4013692, 8.7839815), (49.4014522, 8.7830784), (49.4016043, 8.7818422), (49.4017995, 8.7804795), (49.4020178, 8.7792845), (49.4022397, 8.7783449), (49.4011574, 8.7779251), (49.3998965, 8.7772971), (49.3989502, 8.776723), (49.3982255, 8.7762703), (49.3974986, 8.7758527), (49.3969415, 8.7756097), (49.3963526, 8.7754055), (49.3958899, 8.7753513), (49.3954984, 8.7753568), (49.39513, 8.775432), 
               (49.3947778, 8.7756125), (49.3944623, 8.7758385), (49.394241, 8.7761026), (49.394054, 8.7764718), (49.3935468, 8.7760111), (49.3930526, 8.7751785), (49.3924566, 8.7758406), (49.392035, 8.7753748), (49.3916537, 8.775114), (49.3912093, 8.7745938), (49.3909791, 8.7742285), (49.3908245, 8.7738779), (49.3905232, 8.7736595), (49.3900552, 8.7734087), (49.3896734, 8.7734628), (49.3893918, 8.7736609), (49.3892126, 8.773016), 
               (49.3888767, 8.7729409), (49.3886671, 8.7726774), (49.3884082, 8.7718432), (49.3881213, 8.7713308), (49.3876506, 8.7701457), (49.3872537, 8.7697868), (49.3871404, 8.7693103), (49.3869744, 8.7689455), (49.386845, 8.7685179), (49.3867885, 8.7681781), (49.3868648, 8.7677585), (49.3870624, 8.7672346), (49.387178, 8.7666542), (49.3874983, 8.7642305), (49.3879515, 8.7631917), (49.387516, 8.7628994), (49.3872486, 8.7629785), 
               (49.3869015, 8.7629905), (49.3864383, 8.7628867), (49.385993, 8.7629089), (49.3857622, 8.7629777), (49.3854512, 8.7631578), (49.3852176, 8.7633804), (49.384783, 8.7637348), (49.3841835, 8.7642559), (49.3833889, 8.7652795), (49.3826998, 8.7659436), (49.3820165, 8.766279), (49.3813415, 8.7668453), (49.3806845, 8.7675163), (49.3796115, 8.768381), (49.3791926, 8.7688298), (49.3790267, 8.7684898), (49.3786082, 8.766754), 
               (49.3785391, 8.7661247), (49.3785178, 8.7653695), (49.3785424, 8.7636606), (49.3785322, 8.7630837), (49.3784444, 8.7626149), (49.3780243, 8.7605438), (49.377905, 8.7597863), (49.3778079, 8.7591865), (49.3778069, 8.7591383), (49.3779433, 8.7589034), (49.3778985, 8.7584861), (49.3779064, 8.7579499), (49.3779222, 8.7569057), (49.3779098, 8.7561979), (49.3778406, 8.7554303), (49.3777662, 8.7548762), (49.3777233, 8.7547183), 
               (49.3775725, 8.7547667), (49.3773566, 8.7542262), (49.3770908, 8.7535417), (49.3768928, 8.75318), (49.3763712, 8.7524804), (49.3761665, 8.7519855), (49.3758528, 8.7512869), (49.3754388, 8.7504861), (49.3752336, 8.7502541), (49.3748598, 8.7499794), (49.3742686, 8.7497843), (49.374157, 8.749682), (49.3738739, 8.749506), (49.3738093, 8.7493491), (49.3737876, 8.7490308), (49.373462, 8.7480366), (49.3734027, 8.7478032), 
               (49.3732987, 8.7475313), (49.3731475, 8.7468817), (49.37295, 8.7455713), (49.3727954, 8.7441413), (49.3731078, 8.7429628), (49.3739238, 8.7414683), (49.3744415, 8.7407289), (49.3745529, 8.7400492), (49.3744909, 8.7384154), (49.3741606, 8.7375645), (49.3733236, 8.7363191), (49.3729555, 8.7357242), (49.372675, 8.7339292), (49.3722236, 8.7323275), (49.372716, 8.7314369), (49.3733109, 8.730507), (49.3732518, 8.7303849), 
               (49.3731793, 8.7301548), (49.3740094, 8.7291382), (49.3748214, 8.72822), (49.3755537, 8.7274039), (49.3739564, 8.7248409), (49.3737472, 8.7244791), (49.3734956, 8.7244013), (49.3722497, 8.7247291), (49.3712113, 8.7250093), (49.3702199, 8.725576), (49.3700659, 8.7253571), (49.3694549, 8.7262592), (49.3686816, 8.7283599), (49.3688172, 8.7285406), (49.368208, 8.7299291), (49.3679334, 8.7304042), (49.3668967, 8.7289416), 
               (49.3667046, 8.7286706), (49.3664412, 8.7284458), (49.3655087, 8.7277172), (49.3641325, 8.7274233), (49.362889, 8.7272017), (49.3619886, 8.7271129), (49.3603743, 8.7260938), (49.3600452, 8.7260553), (49.3594762, 8.7260155), (49.358803, 8.7255991), (49.3586631, 8.7254976), (49.3583083, 8.7252405), (49.3579211, 8.7249365), (49.3576018, 8.724196), (49.3574388, 8.723668), (49.3572533, 8.7223891), (49.3571727, 8.721461), 
               (49.3571919, 8.7212668), (49.3572835, 8.7201217), (49.3573024, 8.7197099), (49.3570652, 8.7194716), (49.3567616, 8.7193418), (49.3563511, 8.7189597), (49.3563789, 8.718652), (49.3563422, 8.7163959), (49.35624, 8.7159867), (49.3559508, 8.7150837), (49.3557463, 8.71423), (49.3556945, 8.7133905), (49.355659, 8.7125124), (49.3556183, 8.7119459), (49.3555249, 8.7116833), (49.3553404, 8.7113998), (49.3552103, 8.7112492), 
               (49.3551102, 8.7108887), (49.3555424, 8.7102703), (49.3565799, 8.70964), (49.3563411, 8.7083024), (49.3561232, 8.7068817), (49.3560421, 8.7063527), (49.3559195, 8.7057764), (49.3558707, 8.7054043), (49.3558484, 8.7052338), (49.3557862, 8.7046665), (49.3557454, 8.7039793), (49.3557223, 8.7033211), (49.3557158, 8.7024902), (49.3558153, 8.7018473), (49.3558457, 8.7017688), (49.3560633, 8.7012064), (49.3563271, 8.7005404), 
               (49.3565504, 8.7000915), (49.3565712, 8.70005), (49.3568675, 8.6994679), (49.3570033, 8.6992243), (49.3570051, 8.6991332), (49.3570045, 8.699042), (49.3574206, 8.6982344), (49.3580392, 8.6969526), (49.3577892, 8.6966863), (49.3577489, 8.6963999), (49.3582385, 8.6944442), (49.3585367, 8.6946209), (49.3586027, 8.69466), (49.3586788, 8.6943685), (49.3583571, 8.6941355), (49.3584801, 8.693654), (49.3583714, 8.6936269), 
               (49.3585068, 8.6929042), (49.3583862, 8.6928745), (49.3585124, 8.6924382), (49.3584145, 8.6923843), (49.3584921, 8.6913995), (49.358787, 8.6914693), (49.3588992, 8.6906837), (49.3589292, 8.6902879), (49.3589418, 8.6892133), (49.3588997, 8.689029), (49.3588594, 8.6872368), (49.3591384, 8.6871857), (49.3597597, 8.6870666), (49.3598242, 8.6870596), (49.3597807, 8.6843777), (49.3598165, 8.6842473), (49.3599072, 8.6820136), 
               (49.3603891, 8.6820844), (49.3604254, 8.6799211), (49.3604268, 8.6798173), (49.3604415, 8.6787494), (49.3604274, 8.6782273), (49.360356, 8.6772483), (49.3602212, 8.6759553), (49.3606677, 8.6758339), (49.361218, 8.6741996), (49.361521, 8.6724424), (49.3605945, 8.6721886), (49.3591865, 8.6719045), (49.3578178, 8.6716797), (49.3578048, 8.6714646), (49.3580033, 8.6698071), (49.3581268, 8.6687636), (49.3581106, 8.6685429), 
               (49.3580646, 8.66801), (49.3579798, 8.6670289), (49.3575926, 8.664876), (49.3575483, 8.6645227), (49.357496, 8.6642566), (49.3571394, 8.6643429), (49.356424, 8.6645159), (49.3545919, 8.6648624), (49.3537693, 8.665018), (49.3536627, 8.664184), (49.3535852, 8.6635784), (49.3543955, 8.6632027), (49.3543138, 8.6624985), (49.3540661, 8.6615291), (49.3539883, 8.6611864), (49.3538501, 8.6605535), (49.3548306, 8.6606616), 
               (49.3548636, 8.6600989), (49.3549377, 8.6588739), (49.3551949, 8.6572433), (49.3543595, 8.6566887), (49.3540174, 8.6563616), (49.3537317, 8.656818), (49.353355, 8.6574037), (49.3533027, 8.6573057), (49.3527603, 8.6579381), (49.3525646, 8.6581468), (49.352578, 8.6580041), (49.3526095, 8.6578029), (49.3526217, 8.6577019), (49.3526351, 8.6575934), (49.3527772, 8.656788), (49.3530211, 8.6553187), (49.3531457, 8.6545408), 
               (49.3532147, 8.6540527), (49.3532448, 8.6537417), (49.3532581, 8.6534741), (49.3532768, 8.6531243), (49.3532787, 8.6528437), (49.3532725, 8.652615), (49.3532461, 8.6523475), (49.353203, 8.6520457), (49.3531912, 8.6518299), (49.3531877, 8.6516357), (49.3532041, 8.6514544), (49.3532543, 8.6512772), (49.3545798, 8.6521038), (49.3552873, 8.6525441), (49.3557702, 8.6529873), (49.356263, 8.6533615), (49.3561124, 8.6545613), 
               (49.35643, 8.6547872), (49.3565583, 8.6548785), (49.3572979, 8.6553542), (49.3581381, 8.6558141), (49.3592904, 8.6564537), (49.3601791, 8.6570089), (49.3617759, 8.6580514), (49.3625711, 8.6591635), (49.3628365, 8.6586508), (49.3630543, 8.6581846), (49.3632772, 8.6577635), (49.3628128, 8.6571423), (49.363115, 8.6563768), (49.3634728, 8.655407), (49.3637372, 8.6547681), (49.3640197, 8.6541115), (49.364273, 8.6535185), 
               (49.3645492, 8.6529145), (49.3649022, 8.6521969), (49.3650935, 8.6518466), (49.3649756, 8.6516913), (49.3659782, 8.6499424), (49.3662698, 8.6501153), (49.3665729, 8.6502809), (49.3669839, 8.6505286), (49.3674777, 8.6508412), (49.3676851, 8.6504907), (49.368118, 8.6497748), (49.368515, 8.6491474), (49.3688557, 8.6486155), (49.3691556, 8.64813), (49.3693948, 8.6477754), (49.3697114, 8.647363), (49.3704642, 8.6463644), 
               (49.3703028, 8.6462064), (49.3700858, 8.6459446), (49.3698569, 8.6456093), (49.3696208, 8.6452463), (49.3694587, 8.6449868), (49.3693239, 8.6447163), (49.3690714, 8.6440349), (49.3686547, 8.642835), (49.3696196, 8.6409498), (49.3699773, 8.640274), (49.3685349, 8.639672), (49.3670998, 8.6391047), (49.3663797, 8.6387968), (49.3653225, 8.6383623), (49.3643574, 8.6379856), (49.3636099, 8.6376641), (49.3631909, 8.6374428), 
               (49.3628043, 8.6373038), (49.3626352, 8.637264), (49.36199, 8.6371356), (49.3610373, 8.6368882), (49.3605378, 8.636737), (49.3603567, 8.6366565), (49.359671, 8.6361028), (49.3587647, 8.635396), (49.3580954, 8.6348595), (49.3576856, 8.6345034), (49.3574451, 8.6344207), (49.3571263, 8.6342869), (49.3568077, 8.634167), (49.3564816, 8.633974), (49.3561005, 8.6337574), (49.3557811, 8.6335502), (49.3554893, 8.633367), 
               (49.3550117, 8.6333554), (49.3539394, 8.6332888), (49.3528379, 8.6332335), (49.3528362, 8.6331116), (49.3528456, 8.6329026), (49.3528395, 8.6326053), (49.3528158, 8.6322273), (49.3528086, 8.6320021), (49.3527963, 8.6319092), (49.3527735, 8.631513), (49.3527338, 8.6307873), (49.3520712, 8.630765), (49.3521053, 8.62964), (49.3521083, 8.629121), (49.3521019, 8.6285659), (49.3520906, 8.6279892), (49.3520028, 8.6256253), 
               (49.3520391, 8.6252141), (49.3521186, 8.625019), (49.3528794, 8.6233231), (49.353877, 8.6210851), (49.3540073, 8.6206854), (49.3540376, 8.6204839), (49.3540732, 8.6202265), (49.3554731, 8.6214367), (49.3556858, 8.6216205), (49.3574167, 8.6231417), (49.35757, 8.6234056), (49.3590632, 8.6244658), (49.3591909, 8.6236269), (49.3594703, 8.6224363), (49.35959, 8.6220894), (49.3597126, 8.6218229), (49.3599157, 8.6213638), 
               (49.3600439, 8.6210861), (49.3602057, 8.6206252), (49.3604606, 8.6199753), (49.3606532, 8.6194432), (49.3608491, 8.6189565), (49.361201, 8.6182365), (49.3615292, 8.6175911), (49.3617672, 8.6170433), (49.3620581, 8.6163501), (49.362414, 8.6154443), (49.3631028, 8.6136373), (49.3636618, 8.6121291), (49.3638492, 8.6115552), (49.3640529, 8.6109946), (49.3641791, 8.6106049), (49.3642507, 8.6103539), (49.3642994, 8.6101279), 
               (49.3643606, 8.6097966), (49.3644178, 8.609518), (49.3644612, 8.6092398), (49.3645257, 8.6089749), (49.3645885, 8.6087625), (49.3646761, 8.6085214), (49.3647789, 8.6082304), (49.3648574, 8.6080034), (49.3649318, 8.6077873), (49.3649624, 8.607562), (49.3650038, 8.6071545), (49.3651587, 8.6058855), (49.3652141, 8.6054789), (49.3654187, 8.6043072), (49.3655768, 8.6043931), (49.366203, 8.6010036), (49.3683765, 8.6032101), 
               (49.3702714, 8.6051675), (49.3700968, 8.6057977), (49.3718857, 8.6065358), (49.3738506, 8.6073324), (49.3742901, 8.6074839), (49.3749861, 8.6072224), (49.3761634, 8.6068102), (49.3767512, 8.6068825), (49.3769718, 8.6073417), (49.3772099, 8.6079161), (49.3775051, 8.6086779), (49.3776494, 8.609066), (49.3777564, 8.6094025), (49.37785, 8.6097569), (49.3779193, 8.6101858), (49.3780169, 8.6108659), (49.3782023, 8.6125814), 
               (49.3785282, 8.6137508), (49.3787578, 8.6145451), (49.3789184, 8.6152939), (49.379158, 8.6161546), (49.3792829, 8.6179315), (49.3800106, 8.6211117), (49.3806697, 8.6243818), (49.3816676, 8.6237852), (49.3819903, 8.6242405), (49.3823871, 8.6248779), (49.382699, 8.625383), (49.3830682, 8.6259858), (49.3833504, 8.6264596), (49.383729, 8.6271218), (49.3841304, 8.6278011), (49.3844446, 8.6283658), (49.3847063, 8.6288504), 
               (49.3849446, 8.6294358), (49.3851072, 8.629858), (49.3852005, 8.6301985), (49.3858991, 8.6334012), (49.386227, 8.6334339), (49.3868974, 8.6335417), (49.3871737, 8.6335983), (49.3871846, 8.633683), (49.387313, 8.6349262), (49.3870397, 8.6366041), (49.3865726, 8.6380862), (49.3864106, 8.6386619), (49.3864133, 8.6393871), (49.3864343, 8.6396666), (49.3864979, 8.639916), (49.3868407, 8.6410376), (49.387149, 8.6420906), 
               (49.3876092, 8.6438065), (49.3880443, 8.6453569), (49.3885366, 8.6475002), (49.389866, 8.6463596), (49.3908844, 8.6455348), (49.3931597, 8.6435266), (49.3931943, 8.6435117), (49.3933813, 8.64343), (49.3930209, 8.6407216), (49.3927461, 8.638521), (49.3927198, 8.638295), (49.3938127, 8.6381854), (49.3956477, 8.6380084), (49.3988001, 8.6375494), (49.4011409, 8.6372219), (49.4025959, 8.637017), (49.4032678, 8.6369263), 
               (49.4034618, 8.6369), (49.4039689, 8.636827), (49.4045515, 8.6367432), (49.4048058, 8.6367059), (49.4050725, 8.6366668), (49.4055659, 8.6365905), (49.406041, 8.6364652), (49.4063905, 8.6363792), (49.4069569, 8.636215), (49.4072424, 8.6361362), (49.40753, 8.6360332), (49.4079319, 8.6358731), (49.4090923, 8.6354073), (49.4107628, 8.634737), (49.4114867, 8.6344463), (49.4124487, 8.6340602), (49.4148068, 8.6330749), 
               (49.4149279, 8.633025), (49.4149614, 8.632938), (49.4150347, 8.6327478), (49.4150072, 8.6327068), (49.4145831, 8.632078), (49.4138334, 8.6309584), (49.4131756, 8.6300342), (49.4124857, 8.6290718), (49.4122128, 8.6285902), (49.4120323, 8.6282626), (49.4113406, 8.6269795), (49.4099575, 8.6244983), (49.4090492, 8.6228506), (49.4089757, 8.6227217), (49.4089751, 8.6227201), (49.4089525, 8.6226483), (49.4090095, 8.6226269), 
               (49.409046, 8.6225529), (49.4073832, 8.6166062), (49.4062033, 8.6124372), (49.4054668, 8.6114613), (49.4059494, 8.6106733), (49.407045, 8.6087941), (49.4084558, 8.6052003), (49.4090794, 8.6036384), (49.4091045, 8.6035403), (49.4091271, 8.603393), (49.4095265, 8.6005224), (49.4097836, 8.5986401), (49.4097994, 8.5984789), (49.409869, 8.5978166), (49.4099478, 8.5971086), (49.4100128, 8.5964568), (49.41008, 8.5957175), 
               (49.4101314, 8.5950271), (49.410194, 8.5942949), (49.4102853, 8.5929914), (49.4103161, 8.5925565), (49.4103254, 8.5924903), (49.4103918, 8.5925039), (49.4105506, 8.5906643), (49.4110409, 8.5848925), (49.4110425, 8.5848405), (49.4108959, 8.5847291), (49.4108089, 8.5846596), (49.4107403, 8.5846004), (49.4105983, 8.5844612), (49.4104104, 8.5842557), (49.4106036, 8.5836206), (49.4108304, 8.5828103), (49.4110212, 8.582126), 
               (49.4111982, 8.5814665), (49.4113138, 8.5810212), (49.4114249, 8.5805476), (49.4114771, 8.5803266), (49.4118155, 8.5788706), (49.4121622, 8.5774183), (49.4123091, 8.5768058), (49.4124034, 8.5763544), (49.4124793, 8.5759519), (49.4126195, 8.5751259), (49.4126586, 8.5749054), (49.4144215, 8.574098), (49.4147232, 8.5739798), (49.4157339, 8.573622), (49.4157627, 8.5734302)]

# centroid of the Heidelberg polygon border
poly_centroid = geometry.Point(8.694361786680039, 49.40548948222444)
    
# create folium map  
plot_map = folium.Map(tiles='stamenterrain',location=(poly_centroid.y, poly_centroid.x), zoom_start=12) 
# add company location to map
folium.Marker(locations[0, ::-1], popup='<strong>Our Company (Depot):</strong> {}'.format(names[0]), \
              icon=folium.Icon(color='blue',icon_color='black', icon='home' , prefix='fa')).add_to(plot_map) 
# add customer location to map
for counter, point in enumerate(locations[1:]):
    folium.Marker(point[::-1], popup='<strong>Node {}:</strong> {} <br>demand: {}'.format(counter+1, names[counter+1], demands[counter+1]), \
                  icon=folium.Icon(color='green', icon_color='black', icon='child', prefix='fa')).add_to(plot_map)
# add Heidelberg polygon to map
folium.features.PolygonMarker(poly_coords, popup='Heidelberg Polygon', color='#000000', fill_color='#140e8c', \
                              fill_opacity=0.2, weight=3).add_to(plot_map) 
# show map
display(plot_map)

The blue marker shows the location of our company and the green markers are the locations of the customers. Clicking on the markers will show the node number, address and demand.

# Solving the logistic problem

First, we use the [**matrix API**](https://openrouteservice.org/documentation/#/reference/matrix/matrix), which will give us a symmetric duration matrix for a list of locations/coordinates, where every location is paired with each other. The values of the matrix are given in seconds. 

In [4]:
api_key = '58d904a497c67e00015b45fc9298e8d961e64b48b066a43e51d39887'
client = openrouteservice.Client(key=api_key)

response = client.distance_matrix(locations=locations.tolist(), metrics=['duration'])
duration_matrix = np.array(response['durations']).astype(int)
df_duration_matrix = pd.DataFrame(duration_matrix)
display(df_duration_matrix.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,147,635,383,515,392,561,423,404,248,...,338,617,662,703,250,643,307,487,296,659
1,147,0,523,271,403,280,513,344,292,136,...,226,505,550,590,138,531,195,375,184,547
2,585,473,0,304,699,438,652,510,400,416,...,300,366,496,541,434,781,341,530,377,408
3,412,300,251,0,526,265,609,337,277,243,...,127,439,535,445,261,604,168,388,204,480
4,530,418,755,503,0,512,575,576,524,337,...,458,737,782,609,298,264,427,393,416,779


As we can see the openrouteservice API returns a symmetric duration matrix with the diagonal being zeros, since the duration to get from every node to itself is zero. 

The SetArcCostEvaluatorOfAllVehicles() and AddDimension() method of the or-tool package requires a callable object. Therefore we have to wrap the duration matrix and the demands within a class with suitable methods that can be called by named methods. 

In [5]:
class matrix(object): 
    """Creates callback to return duration between points."""
    def __init__(self, matrix):
        """Initializes the duration matrix."""
        self._matrix = matrix

    def Duration(self, from_node, to_node):
        """Returns the duration between the two nodes"""
        return self._matrix[from_node][to_node]

class demand(object): 
    """Creates callback to get demands at each location."""
    def __init__(self, demands):
        """Initializes the demand array."""
        self._demands = demands

    def Demand(self, from_node, to_node):
        """Returns the demand of the current node"""
        del to_node
        return self._demands[from_node]

In the next step, we use the or-tool to solve the problem with our obtained duration matrix and demand modified with suitable callbacks:

In [6]:
# Create Routing Model
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)

# Define weight of each edge
duration_evaluator = matrix(duration_matrix).Duration
routing.SetArcCostEvaluatorOfAllVehicles(duration_evaluator)

# Add Capacity constraint
demand_evaluator = demand(demands).Demand
routing.AddDimension(demand_evaluator, 0, capacity, True, "Capacity") 

# Setting first solution heuristic (cheapest addition).
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem
assignment = routing.SolveWithParameters(search_parameters)

# Print the solution
total_dur = 0
routes = []
for vehicle_id in range(num_vehicles):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
    route_dur = 0
    route_load = 0
    temp_route = []
    while not routing.IsEnd(index):
        node_index = routing.IndexToNode(index)
        next_node_index = routing.IndexToNode(assignment.Value(routing.NextVar(index)))
        route_dur += matrix(duration_matrix).Duration(node_index, next_node_index)
        route_load += demands[node_index]
        temp_route.append(node_index)
        plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
        index = assignment.Value(routing.NextVar(index))
    node_index = routing.IndexToNode(index)
    total_dur += route_dur
    temp_route.append(node_index)
    routes.append(temp_route)
    plan_output += ' {0} Load({1})\n'.format(node_index, route_load)
    plan_output += 'Duration of the route: {0:.2f}min\n'.format(route_dur / 60)
    plan_output += 'Load of the route: {0}\n'.format(route_load)
    print(plan_output)

Route for vehicle 0:
 0 Load(0) ->  6 Load(11) ->  19 Load(15) ->  17 Load(40) ->  4 Load(49) ->  16 Load(75) ->  0 Load(75)
Duration of the route: 32.80min
Load of the route: 75

Route for vehicle 1:
 0 Load(0) ->  7 Load(22) ->  14 Load(35) ->  21 Load(46) ->  2 Load(52) ->  15 Load(56) ->  10 Load(79) ->  0 Load(79)
Duration of the route: 51.87min
Load of the route: 79

Route for vehicle 2:
 0 Load(0) ->  8 Load(18) ->  13 Load(46) ->  11 Load(67) ->  5 Load(86) ->  0 Load(86)
Duration of the route: 22.28min
Load of the route: 86

Route for vehicle 3:
 0 Load(0) ->  9 Load(6) ->  12 Load(9) ->  3 Load(29) ->  18 Load(31) ->  20 Load(46) ->  1 Load(70) ->  0 Load(70)
Duration of the route: 15.80min
Load of the route: 70



For some visualisation, we can plot the 4 routes with folium using the openrouteservice [**direction API**](https://openrouteservice.org/documentation/#/reference/directions):

In [7]:
# add 4 route to map using the direction API
color = ['red', 'green', 'blue', 'black']
for counter, route in enumerate(routes):
    points = locations[route]
    response = client.directions(coordinates=points, geometry_format='polyline')
    route = np.array(response['routes'][0]['geometry'])
    folium.PolyLine(list(zip(route[:, ::-1].T[0], route[:, ::-1].T[1])), color=color[counter]).add_to(plot_map)

# show map
display(plot_map)

In [8]:
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

d = style_function('red')
print(d)

<function style_function.<locals>.<lambda> at 0x0000000009295950>


In [9]:
print(locations)

[[  8.683448  49.416961]
 [  8.681247  49.412388]
 [  8.662789  49.37446 ]
 [  8.663488  49.390504]
 [  8.650365  49.424738]
 [  8.684574  49.393819]
 [  8.636446  49.430018]
 [  8.693395  49.401669]
 [  8.685109  49.387783]
 [  8.670162  49.406377]
 [  8.63245   49.409997]
 [  8.689162  49.372064]
 [  8.670682  49.396678]
 [  8.690306  49.372096]
 [  8.69358   49.378182]
 [  8.63606   49.395647]
 [  8.670398  49.408443]
 [  8.637578  49.429881]
 [  8.682342  49.403657]
 [  8.64455   49.413006]
 [  8.685483  49.406815]
 [  8.691885  49.372719]]


In [10]:
print(locations[routes[0]])

[[  8.683448  49.416961]
 [  8.636446  49.430018]
 [  8.64455   49.413006]
 [  8.637578  49.429881]
 [  8.650365  49.424738]
 [  8.670398  49.408443]
 [  8.683448  49.416961]]


In [11]:
print(locations[routes[0], ::-1])

[[ 49.416961   8.683448]
 [ 49.430018   8.636446]
 [ 49.413006   8.64455 ]
 [ 49.429881   8.637578]
 [ 49.424738   8.650365]
 [ 49.408443   8.670398]
 [ 49.416961   8.683448]]


In [12]:
d = locations[routes[0], ::-1]
print(d.T)
print(list(zip(d.T[0], d.T[1])))

[[ 49.416961  49.430018  49.413006  49.429881  49.424738  49.408443
   49.416961]
 [  8.683448   8.636446   8.64455    8.637578   8.650365   8.670398
    8.683448]]
[(49.416961000000001, 8.6834480000000003), (49.430017999999997, 8.6364459999999994), (49.413006000000003, 8.6445500000000006), (49.429881000000002, 8.6375779999999995), (49.424737999999998, 8.6503650000000007), (49.408442999999998, 8.6703980000000005), (49.416961000000001, 8.6834480000000003)]


In [13]:
help(folium.PolyLine)

Help on class PolyLine in module folium.features:

class PolyLine(folium.map.Marker)
 |  Creates a PolyLine (array) or MultiPolyline (array of arrays) object to
 |  append into a map.
 |  
 |  Parameters
 |  ----------
 |  locations: list of points (latitude, longitude)
 |      Latitude and Longitude of line (Northing, Easting)
 |  color: string, default Leaflet's default ('#03f')
 |  weight: float, default Leaflet's default (5)
 |  opacity: float, default Leaflet's default (0.5)
 |  popup: string or folium.Popup, default None
 |      Input text or visualization for object.
 |  
 |  See http://leafletjs.com/reference-1.2.0.html#polyline for more options.
 |  
 |  Method resolution order:
 |      PolyLine
 |      folium.map.Marker
 |      branca.element.MacroElement
 |      branca.element.Element
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, locations, popup=None, **kw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  -----